In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [66]:
import pandas as pd
df = pd.read_csv("princess_corpus.csv", skipinitialspace=True)
df.head()


,Disney_Period,Text,Speaker_Status,Movie,Speaker,Year,UTTERANCE_NUMBER
0,EARLY,slave in the magic mirror come from the farthe...,NON-P,Snow White,queen,1937,1
1,EARLY,"what wouldst thou know, my queen ?",NON-P,Snow White,mirror,1937,2
2,EARLY,"magic mirror on the wall, who is the fairest o...",NON-P,Snow White,queen,1937,3
3,EARLY,"famed is thy beauty, majesty. but hold, a love...",NON-P,Snow White,mirror,1937,4
4,EARLY,alas for her ! reveal her name.,NON-P,Snow White,queen,1937,5


In [67]:
# import nltk
# nltk.download('corpus')


[nltk_data] Error loading corpus: Package 'corpus' not found in index


False

In [68]:
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk import wordpunct_tokenize
morph = MorphAnalyzer()
stops = set(stopwords.words('english'))


def lemmatize(x):
    if type(x) != str:
        return ""
    text = wordpunct_tokenize(x)
    result = []
    for word in text:
        if word.isalpha():
            nf = morph.parse(word)[0].normal_form
            #if nf not in stops:
            result.append(nf)
    return " ".join(result)


In [69]:
from pymystem3 import Mystem
m = Mystem()


def parse(x):
    result = []
    ana = nltk.pos_tag(x.split())
    for tagged_token in ana:
        result.append(tagged_token[1])
    return ' '.join(result)


def length(x):
    return len(x)


In [70]:
df['Length'] = (df['Text']).apply(length)
df['Text'] = df['Text'].apply(lemmatize)


In [71]:
df['POS'] = df['Text'].apply(parse)
df


,Disney_Period,Text,Speaker_Status,Movie,Speaker,Year,UTTERANCE_NUMBER,Length,POS
0,EARLY,slave in the magic mirror come from the farthe...,NON-P,Snow White,queen,1937,1,125,NN IN DT JJ NN VBN IN DT JJS NN IN NN CC NN NN...
1,EARLY,what wouldst thou know my queen,NON-P,Snow White,mirror,1937,2,35,WP VBZ NN VB PRP$ NN
2,EARLY,magic mirror on the wall who is the fairest on...,NON-P,Snow White,queen,1937,3,58,JJ NN IN DT NN WP VBZ DT JJS CD IN DT
3,EARLY,famed is thy beauty majesty but hold a lovely ...,NON-P,Snow White,mirror,1937,4,130,VBN VBZ JJ NN NN CC VBP DT JJ NN NN VBP NNS VB...
4,EARLY,alas for her reveal her name,NON-P,Snow White,queen,1937,5,32,NN IN PRP$ NN PRP$ NN
...,...,...,...,...,...,...,...,...,...
7743,LATE,we are never closing them again,PRINCESS,Frozen,elsa,2013,984,33,PRP VBP RB VBG PRP RB
7744,LATE,form on anna s boots,PRINCESS,Frozen,elsa,2013,985,21,NN IN NN NN NNS
7745,LATE,what oh elsa they re beautiful but you know i ...,PRINCESS,Frozen,anna,2013,986,60,WP VBZ NN PRP VBP JJ CC PRP VBP JJ VBP JJ NN
7746,LATE,look out reindeer coming through,NON-P,Frozen,kristoff,2013,987,35,VB RP NN VBG IN


In [72]:
mean = df['Length'].mean()
mean


51.59112028910687

In [73]:
# this is basically a replacement for tf-idf, i just need to get this thing working rn
from sklearn.feature_extraction.text import CountVectorizer


In [74]:
df['Speaker_Status'] = df['Speaker_Status'].map(
    {'PRINCE': 1, 'PRINCESS': 1, 'NON-P': 0})
df['Speaker_Status'].head(20)


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    1
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Speaker_Status, dtype: int64

In [75]:
df.groupby("Speaker_Status")["Text"].count()


Speaker_Status
0    4925
1    2823
Name: Text, dtype: int64

In [76]:
df_half = df[:200]
df_half.groupby("Speaker_Status")["Text"].count()


Speaker_Status
0    145
1     55
Name: Text, dtype: int64

In [77]:
texts = df["Text"]
y = df["Speaker_Status"]
data_train, data_test, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, random_state=0)


In [78]:
# cvec = CountVectorizer(binary=True, max_features=10000)
# cvec.fit(data_train)

In [79]:
from sklearn.feature_extraction.text import TfidfTransformer


In [80]:
tfIdfTransformer = TfidfTransformer(use_idf=True)
countVectorizer = CountVectorizer()
wordCount = countVectorizer.fit_transform(df["Text"])
newTfIdf = tfIdfTransformer.fit_transform(wordCount)
dfnew = pd.DataFrame(newTfIdf[0].T.todense(
), index=countVectorizer.get_feature_names(), columns=["TF-IDF"])
dfnew = dfnew.sort_values('TF-IDF', ascending=False)
print(dfnew.head(25))


              TF-IDF
summon      0.301476
farthest    0.301476
space       0.278915
thy         0.271652
darkness    0.271652
slave       0.265717
thee        0.265717
mirror      0.252520
speak       0.245988
wind        0.226529
face        0.214435
magic       0.200134
through     0.181816
the         0.171521
from        0.161382
let         0.148926
see         0.146057
come        0.142668
in          0.118773
me          0.114494
and         0.101981
process     0.000000
projection  0.000000
progress    0.000000
private     0.000000


/home/chenopodium/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [94]:
# X_train = tfIdfTransformer.fit_transform(data_train)
# X_test = tfIdfTransformer.fit_transform(data_test)
X_train, X_test, y_train, y_test = train_test_split(
    texts, y, random_state=0, test_size=0.2)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)


In [99]:
model = LogisticRegression(class_weight="balanced")

model.fit(X_train, y_train)


ValueError: could not convert string to float: 'who disturbs my slumber'

In [97]:
from sklearn.metrics import accuracy_score, f1_score


In [98]:
model.predict_proba(X_test)[:5].round(2)


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [89]:
model.predict(X_test)

array([0, 0, 1, ..., 0, 1, 0])

In [90]:
y_pred = model.predict(X_test)


In [91]:
accuracy_score(y_test, y_pred)


0.6529032258064517